EL PLANTEAMIENTO CON RESPECTO A LA PRIMERA VERSIÓN ES DIFERENTE:
A LA VISTA DE LOS ERRORES QUE SE PRODUCEN CON LA DESCARGA DEL TOTAL RETURN POR PARTES PRINCIPALMENTE DE LOS ÍNDICES DE RENTA FIJA, LO QUE SE HACE EN ESTE ALGORITMO ES DESCARGAR TODA LA SERIE TEMPORAL DE PRECIOS, ALINEARLAS TEMPORALMENTE Y POSTERIORMENTE CALCULAR LAS CARTERAS Y EL PERFORMANCE.
DE HECHO, ESTA VERSIÓN SE HA PLANTEADO DEBIDO A QUE ALGUNOS ÍNDICES NO ESTÁN REPRESENTADOS EN PORT <GO> Y LOS RESULTADOS QUE SE OBTIENEN NO SON REALES.

In [1]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
import bqviz as bqv
from scipy.optimize import minimize
from dateutil.relativedelta import relativedelta
import bqport
from bqport import PositionType
from sklearn import linear_model

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

equity_idx = ['SPY US Equity', 'EEM US Equity']
fi_idx = ['HYG US Equity', 'IHYG LN Equity', 'IEI US Equity', 'SMTC LN Equity']

etf_list = equity_idx + fi_idx

In [2]:
# INPUTS ###############

index_to_replicate = 'HFRIILAU Index'

start_date = datetime.datetime(2009,1,1)
end_date = datetime.datetime(2019,1,1)
curncy = 'USD'
rotation = 1 #weeks
max_turnover = 0.25

In [3]:
#The fund inception of some ETFs it's too close to crrent days, so to make the analysis we will asume that these ETFs replicate exactly the underlying indexes.
#Due to this hypothesis, the time series are going to be analyzed with the underlying indexes.

etf_under_idx = d.fund_benchmark()

request = bql.Request(u.list(etf_list), {'UND_IDX': etf_under_idx})
res = bq.execute(request)
etf_under_idx_list = [undr +' Index' for undr in list(res[0].df()['UND_IDX'])]

In [4]:
#Optimization functions
def f_obj(x):
#     return sum((np.matmul(A, x) - y)**2)

    sharpe_port = ((np.matmul(A, x) - y)[-1])/np.std(np.matmul(A, x) - y)
    sharpe_bk = y[-1]/np.std(y)
    
    return sharpe_bk - sharpe_port

def constraint_total_weight_ub(x):  #The sum of all weights must be 1
    return 1 - np.sum(x)

def constraint_tracking_error(x):  #tracking Error lower than 1.5%
    return 1.5 - np.std(np.matmul(A, x) - y)*100

def compute_port_vs_bench_difference (df_port, df_group_price_series, df_align_price_series, df_index_price_series):
    
    #Securities don't follow the same order in different time periods
    df_port = df_port.sort_values(by=['date', 'security'])

    #Add to the portfolio df an auxiliar date 
    year_list = pd.DatetimeIndex(df_port['date']).year.tolist()
    month_list = pd.DatetimeIndex(df_port['date']).month.tolist()
    date_list = []
    for j in range(len(year_list)):
        date_list += [str(year_list[j]) + '-' + str(month_list[j])]
    df_port['aux_date'] = date_list  

    #Get the price and consecuently the number of shares for each moment of the rebalancing date
    list_reb_dates = df_port['aux_date'].unique().tolist()

    price_0_list = []
    for i_date in list_reb_dates:
        aux_port_df = df_port.loc[df_port['aux_date'].isin([i_date])]
        price_0_list += df_group_price_series.loc[df_group_price_series['aux_date'].isin([i_date])][aux_port_df['security'].tolist()].values[0].tolist()
    df_port['Price'] = price_0_list
    df_port['N_Assets_type'] = df_port['weight']/df_port['Price']

    #For the first date the calculated number of shares it's OK, but for the rest of the dates it must be take into account the value of the portfolio in that moment
    aux_df_real_n_assets = df_port.loc[df_port['aux_date'].isin([list_reb_dates[0]])][['aux_date', 'N_Assets_type']]

    for i_prev_date,i_date in zip(list_reb_dates[:-1], list_reb_dates[1:]):
        n_assets = np.array(aux_df_real_n_assets.loc[aux_df_real_n_assets['aux_date'].isin([i_prev_date])]['N_Assets_type'].tolist())
        cur_price = np.array(df_port.loc[df_port['aux_date'].isin([i_date])]['Price'].tolist())
        val = sum(n_assets*cur_price)
        cur_n_assets = val*np.array(df_port.loc[df_port['aux_date'].isin([i_date])]['weight'].tolist())/cur_price

        aux_df_real_n_assets = aux_df_real_n_assets.append(pd.DataFrame(data={'aux_date': [i_date]*len(cur_n_assets), 'N_Assets_type': cur_n_assets}), ignore_index = True)

    df_port['Real_N_assets'] = aux_df_real_n_assets['N_Assets_type'].tolist()

    #Put the portfolio df with the number of shares in each period of time in the same format as the price series df
    port_df_pivot = df_port.pivot(index = 'aux_date', columns = 'security', values = 'Real_N_assets').reset_index()

    n_assets_type_table = pd.merge(port_df_pivot, df_align_price_series['aux_date'].to_frame(), how='left', on=['aux_date', 'aux_date']).fillna(method='ffill')

    #Compute the portfolio value
    port_val = df_align_price_series.copy()
    port_val = port_val.loc[port_val['aux_date'].isin(n_assets_type_table['aux_date'].unique())]
    for asset in etf_under_idx_list:
        port_val[asset] = np.array(port_val[asset].tolist()) * np.array(n_assets_type_table[asset].tolist())

    port_val['Total_Abs'] = port_val[etf_under_idx_list].sum(axis=1)

    #Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
    def_series = df_index_price_series.copy()
    def_series = def_series.loc[def_series['aux_date'].isin(n_assets_type_table['aux_date'].unique())]
    aligned_port_vals = []
    for i_date in def_series['aux_date'].tolist():
        aligned_port_vals += [port_val.loc[port_val['aux_date'].isin([i_date])]['Total_Abs'].values[-1]]

    #Calculate the cumulated total return of both value series
    def_series['Port'] = aligned_port_vals
    def_series = def_series.dropna(subset = ['Port'])
    def_series[['Port', index_to_replicate]] = (def_series[['Port', index_to_replicate]].pct_change().fillna(0) + 1).cumprod() - 1

    #Sort Columns
    def_series = def_series[['DATE', 'Port', index_to_replicate, 'aux_date']]

    diff = def_series.iloc[len(def_series)-1]['Port'] - def_series.iloc[len(def_series)-1][index_to_replicate]

    return diff

In [5]:
# #Price Aligneation ####################

# #Get the whole price series por all the indexes of the ETFs
# download_start_date = start_date - relativedelta(months=12)
# download_end_date = end_date + relativedelta(months=rotation)

# price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
# request = bql.Request(u.list(etf_under_idx_list), {'Price': price})
# res = bq.execute(request)
# price_series = res[0].df().reset_index()

# #Pivot the price series to get the df in a way that the rows are dates and columns indexes prices
# align_price_series = price_series.pivot(index='DATE', columns='ID', values='Price').reset_index().dropna().reset_index().drop(['index'], axis=1)

# #Add to the price series df an auxiliar date 
# year_list = pd.DatetimeIndex(align_price_series['DATE']).year.tolist()
# month_list = pd.DatetimeIndex(align_price_series['DATE']).month.tolist()
# date_list = []
# for j in range(len(year_list)):
#     date_list += [str(year_list[j]) + '-' + str(month_list[j])]
# align_price_series['aux_date'] = date_list  

# #Get the price series for HFRIFOF Index (the returned data is in a monthly basis)
# price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
# request = bql.Request(index_to_replicate, {'Price': price})
# res = bq.execute(request)
# index_price_series = res[0].df().reset_index()

# #Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
# index_price_series = index_price_series.drop(['ID', 'CURRENCY'], axis=1)
# index_price_series = index_price_series.rename(index=str, columns={'Price': index_to_replicate})
# year_list = pd.DatetimeIndex(index_price_series['DATE']).year.tolist()
# month_list = pd.DatetimeIndex(index_price_series['DATE']).month.tolist()
# date_list = []
# for j in range(len(year_list)):
#     date_list += [str(year_list[j]) + '-' + str(month_list[j])]
# index_price_series['aux_date'] = date_list

# grouped_price_series = pd.DataFrame(data = {})

# list_aux_dates = align_price_series['aux_date'].unique().tolist()
# for i_date in list_aux_dates:
#     aux_mat = align_price_series.loc[align_price_series['aux_date'].isin([i_date]), align_price_series.columns != 'DATE']
#     i_idx = aux_mat.index.tolist()
#     aux_mat = aux_mat.loc[i_idx[-1]].to_frame().T
    
#     if i_date == list_aux_dates[0]:
#         grouped_price_series = aux_mat
        
#     else:
#         grouped_price_series = grouped_price_series.append(aux_mat, ignore_index=True)

# grouped_price_series = pd.merge(index_price_series, grouped_price_series, how = 'left', on=['aux_date', 'aux_date'])

# index_universe = [index_to_replicate]+etf_under_idx_list

In [6]:
#Load data downloaded whith BDH

data = pd.read_excel('Datos_Analisis_Semanal_BPI_v1_2.xlsx')
grouped_price_series = data.drop_duplicates(subset=['aux_date'], keep = 'last')
index_price_series = grouped_price_series[['DATE', 'HFRIILAU Index', 'aux_date']]
align_price_series = data.loc[:,data.columns != 'HFRIILAU Index']
etf_under_idx_list = data[data.columns[~data.columns.isin(['DATE', 'aux_date', 'HFRIILAU Index'])]].columns.tolist()

In [7]:
#Backtesting ############################

aux_date = start_date
while end_date >= aux_date:
    print(aux_date)
    
    aux_date_st = aux_date - relativedelta(months=12)

#     #Compute the spread between the portfolio and the benchmark. This spread (the cumulative total return to the current point) is going to be applied in the in-sample cumulative total return
#     #of the benchmark to shift the base. This is a way to correct the out-of-sample tracking error
#     try:
#         spread_port_bk = compute_port_vs_bench_difference(port_df, grouped_price_series, align_price_series, index_price_series)
    
#     except:
#         spread_port_bk = 0
    
#     print('spread: ', spread_port_bk)
    
    spread_port_bk = 0
    
    test_data = grouped_price_series.loc[(grouped_price_series['DATE'] >= aux_date_st) & (grouped_price_series['DATE'] <= aux_date)]
    test_data = test_data.loc[:,((grouped_price_series.columns != 'aux_date') & (grouped_price_series.columns != 'DATE')).tolist()]

    #Calculate the return
    test_data = test_data.pct_change().fillna(0)    
    
#     #Calculate the cumulative total return
#     test_data = (test_data.pct_change().fillna(0) + 1).cumprod() - 1    
    
    #Optimzation of series without UCITS constraints to see which is the optimal allocation

    #Apply RANSAC to get rid off the outliers fron the sample
    A = []
    for i in range(len(test_data)):
        A += [test_data[etf_under_idx_list].iloc[i].values.tolist()]
    A = np.array(A)    

    y = test_data[index_to_replicate].tolist()
    y = np.array(y) - spread_port_bk
    
    ransac = linear_model.RANSACRegressor(max_trials=1000)
    ransac.fit(A, y)
    
    test_data = test_data.loc[ransac.inlier_mask_]
    
    #Apply to the filtered sample with RANSAC the optimization
    A = []
    for i in range(len(test_data)):
        A += [test_data[etf_under_idx_list].iloc[i].values.tolist()]
    A = np.array(A)    

    y = test_data[index_to_replicate].tolist()
    y = np.array(y) - spread_port_bk

    guess=np.zeros(len(etf_under_idx_list))

    bnds = [(0,0.5) for i in range(len(etf_under_idx_list))] #Weight interval between 0% - 50%
    constr1 = {'type':'eq','fun' : constraint_total_weight_ub}
    constr2 = {'type':'ineq','fun' : constraint_tracking_error}
    results = minimize(fun = f_obj, x0 = guess, method='SLSQP', bounds = bnds, options={'disp': True}, constraints= [constr1, constr2])  
#     results = minimize(fun = f_obj, x0 = guess, method='SLSQP', bounds = bnds, options={'disp': True}, constraints= [constr1])      
    
    if aux_date == start_date:
        port_df = pd.DataFrame({'date': aux_date,'security': etf_under_idx_list, 'weight': results.x})
    else :
        aux_port_df = pd.DataFrame({'date': aux_date,'security': etf_under_idx_list, 'weight': results.x})
        port_df = pd.concat([port_df, aux_port_df])

    aux_date = aux_date + relativedelta(weeks=rotation)
    
port_df

2009-01-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -3.50905981939
            Iterations: 14
            Function evaluations: 131
            Gradient evaluations: 14
2009-01-08 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.855259293684
            Iterations: 16
            Function evaluations: 152
            Gradient evaluations: 16
2009-01-15 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -3.19748322373
            Iterations: 14
            Function evaluations: 129
            Gradient evaluations: 14
2009-01-22 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -3.22365730884
            Iterations: 15
            Function evaluations: 138
            Gradient evaluations: 15
2009-01-29 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            

date        security        weight
0  2009-01-01  LF98TRUU Index  2.505512e-01
1  2009-01-01  LP01TREU Index  0.000000e+00
2  2009-01-01  LUATTRUU Index  2.050973e-01
3  2009-01-01      MXEF Index  0.000000e+00
4  2009-01-01   BNPIMUS Index  5.000000e-01
5  2009-01-01       SPX Index  0.000000e+00
6  2009-01-01      DXY Curncy  4.435146e-02
0  2009-01-08  LF98TRUU Index  1.821123e-14
1  2009-01-08  LP01TREU Index  8.158834e-02
2  2009-01-08  LUATTRUU Index  7.033864e-17
3  2009-01-08      MXEF Index  1.008178e-01
4  2009-01-08   BNPIMUS Index  5.000000e-01
5  2009-01-08       SPX Index  1.798113e-02
6  2009-01-08      DXY Curncy  2.996127e-01
0  2009-01-15  LF98TRUU Index  0.000000e+00
1  2009-01-15  LP01TREU Index  2.795492e-01
2  2009-01-15  LUATTRUU Index  1.582933e-01
3  2009-01-15      MXEF Index  0.000000e+00
4  2009-01-15   BNPIMUS Index  5.000000e-01
5  2009-01-15       SPX Index  1.420400e-13
6  2009-01-15      DXY Curncy  6.215747e-02
0  2009-01-22  LF98TRUU Index  0.000000e+00
1  2009-01-22  LP01TREU Index  1.012379e-01
2  2009-01-22  LUATTRUU Index  2.285694e-01
3  2009-01-22      MXEF Index  1.883982e-02
4  2009-01-22   BNPIMUS Index  5.000000e-01
5  2009-01-22       SPX Index  0.000000e+00
6  2009-01-22      DXY Curncy  1.513529e-01
0  2009-01-29  LF98TRUU Index  1.298812e-01
1  2009-01-29  LP01TREU Index  4.304676e-15
..        ...             ...           ...
5  2018-11-29       SPX Index  0.000000e+00
6  2018-11-29      DXY Curncy  1.738224e-01
0  2018-12-06  LF98TRUU Index  2.376627e-01
1  2018-12-06  LP01TREU Index  0.000000e+00
2  2018-12-06  LUATTRUU Index  0.000000e+00
3  2018-12-06      MXEF Index  1.142111e-01
4  2018-12-06   BNPIMUS Index  5.000000e-01
5  2018-12-06       SPX Index  0.000000e+00
6  2018-12-06      DXY Curncy  1.481262e-01
0  2018-12-13  LF98TRUU Index  4.042330e-01
1  2018-12-13  LP01TREU Index  0.000000e+00
2  2018-12-13  LUATTRUU Index  5.000000e-01
3  2018-12-13      MXEF Index  9.576701e-02
4  2018-12-13   BNPIMUS Index  3.759303e-14
5  2018-12-13       SPX Index  2.179599e-14
6  2018-12-13      DXY Curncy  0.000000e+00
0  2018-12-20  LF98TRUU Index  2.482525e-01
1  2018-12-20  LP01TREU Index  3.854367e-01
2  2018-12-20  LUATTRUU Index  2.356842e-02
3  2018-12-20      MXEF Index  1.326591e-01
4  2018-12-20   BNPIMUS Index  0.000000e+00
5  2018-12-20       SPX Index  2.047584e-16
6  2018-12-20      DXY Curncy  2.100834e-01
0  2018-12-27  LF98TRUU Index  3.034169e-01
1  2018-12-27  LP01TREU Index  3.387781e-01
2  2018-12-27  LUATTRUU Index  7.468915e-16
3  2018-12-27      MXEF Index  1.668242e-01
4  2018-12-27   BNPIMUS Index  0.000000e+00
5  2018-12-27       SPX Index  6.677543e-16
6  2018-12-27      DXY Curncy  1.909808e-01

[3654 rows x 3 columns]

In [8]:
#TURNOVER LIMITATION #####################

#Securities don't follow the same order in different time periods
port_df = port_df.sort_values(by=['date', 'security'])

list_dates = port_df['date'].unique()
list_dates = [pd.to_datetime(str(dates_str)).strftime('%Y-%m-%d') for dates_str in list_dates]

new_port = port_df.loc[port_df['date'].isin([list_dates[0]])]

for idx in range(1,len(list_dates)):
    prev_weights = np.array(new_port.loc[new_port['date'].isin([list_dates[idx - 1]]), 'weight'].tolist())
    cur_teorical_weights = np.array(port_df.loc[port_df['date'].isin([list_dates[idx]]), 'weight'].tolist())
    best_weights = prev_weights + (cur_teorical_weights - prev_weights)*max_turnover
    
    aux_df_new_port = port_df.loc[port_df['date'].isin([list_dates[idx]])][['date', 'security']]
    aux_df_new_port['weight'] = best_weights
    
    new_port = new_port.append(aux_df_new_port, ignore_index = True)

port_df = new_port

In [12]:
#Add to the portfolio df an auxiliar date 
year_list = pd.DatetimeIndex(port_df['date']).year.tolist()
month_list = pd.DatetimeIndex(port_df['date']).week.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
port_df['aux_date'] = date_list  

#Get the price and consecuently the number of shares for each moment of the rebalancing date
list_reb_dates = port_df['aux_date'].unique().tolist()

aux_auxdate_list = []
aux_sec_list = []
aux_prices_list = []
for i_list in range(len(grouped_price_series[etf_under_idx_list].values)):
    aux_prices_list += grouped_price_series[etf_under_idx_list].values[i_list].tolist()
    aux_sec_list += etf_under_idx_list
    aux_auxdate_list += [grouped_price_series['aux_date'].values[i_list]] * len(etf_under_idx_list)

prices_dataframe = pd.DataFrame(data = {'aux_date': aux_auxdate_list, 'security': aux_sec_list, 'Price': aux_prices_list})
port_df = pd.merge(port_df, prices_dataframe, how = 'left', on = ['aux_date', 'security'])
    
port_df['N_Assets_type'] = port_df['weight']/port_df['Price']

#For the first date the calculated number of shares it's OK, but for the rest of the dates it must be take into account the value of the portfolio in that moment
aux_df_real_n_assets = port_df.loc[port_df['aux_date'].isin([list_reb_dates[0]])][['aux_date', 'N_Assets_type']]
    
for i_prev_date,i_date in zip(list_reb_dates[:-1], list_reb_dates[1:]):
    n_assets = np.array(aux_df_real_n_assets.loc[aux_df_real_n_assets['aux_date'].isin([i_prev_date])]['N_Assets_type'].tolist())
    cur_price = np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['Price'].tolist())
    val = sum(n_assets*cur_price)
    cur_n_assets = val*np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['weight'].tolist())/cur_price
    
    aux_df_real_n_assets = aux_df_real_n_assets.append(pd.DataFrame(data={'aux_date': [i_date]*len(cur_n_assets), 'N_Assets_type': cur_n_assets}), ignore_index = True)

port_df['Real_N_assets'] = aux_df_real_n_assets['N_Assets_type'].tolist()

#Put the portfolio df with the number of shares in each period of time in the same format as the price series df
port_df_pivot = port_df.pivot(index = 'aux_date', columns = 'security', values = 'Real_N_assets').reset_index()
n_assets_type_table = pd.merge(align_price_series['aux_date'].to_frame(), port_df_pivot, how='left', on=['aux_date', 'aux_date']).fillna(method='ffill')

#Compute the portfolio value
port_val = align_price_series.copy()
for asset in etf_under_idx_list:
    port_val[asset] = port_val[asset] * n_assets_type_table[asset]
    
port_val['Total_Abs'] = port_val[etf_under_idx_list].sum(axis=1)

#Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
def_series = index_price_series.copy()

aligned_port_vals = []
for i_date in def_series['aux_date'].tolist():
    aligned_port_vals += [port_val.loc[port_val['aux_date'].isin([i_date])]['Total_Abs'].values[-1]]

#Calculate the cumulated total return of both value series
def_series['Port'] = aligned_port_vals
def_series = def_series.dropna(subset = ['Port'])
def_series[['Port', index_to_replicate]] = (def_series[['Port', index_to_replicate]].pct_change().fillna(0) + 1).cumprod() - 1

#Sort Columns
def_series = def_series[['DATE', 'Port', index_to_replicate, 'aux_date']]

#Plot the result
plot_def_series = def_series.drop(['aux_date'], axis=1).set_index('DATE')
result = bqv.ComparisonPlot(plot_def_series)

# Display the plot
result.show()

In [10]:
# pd.pivot_table(port_df, values='weight', index=['date'],columns=['security'], aggfunc=np.sum).reset_index().to_excel('pesos_cartera_replica_BPI_V1_2.xlsx')
# plot_def_series.to_excel('resultados_replicar_BPI_V1_2.xlsx')